In [5]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [6]:
# 数据库地址：数据库放在上一级目录下
db_path = os.path.join(os.path.dirname(os.getcwd()), "data.db")
engine_path = "sqlite:///" + db_path
# 创建数据库引擎
engine = create_engine(engine_path)

### 获取数据

>user_id：用户id
>
>username：用户名
>
>age：年龄
>
>content：评论内容
>
>sentiment_value：情感值【0消极，1积极，-1未知】（用飞浆重写训练得到情感值）
>
>create_time：评论时间
>
>subject：评论主题

In [18]:
sql = """
select 
a.user_id
,a.username
,a.age
,b.content
--,b.sentiment_value
,b.create_time
,b.subject
from
users as a
left join
comment as b
on a.user_id=b.user_id
"""

df = pd.read_sql(sql, engine)

In [33]:
df.sample(5)

,user_id,username,age,content,sentiment_value,create_time,subject
3779,3780,ruuYYz,45,这不比博人传还燃吗？,0,2021-04-27 00:00:00.000000,其他
6196,6197,RqAKIq,25,强烈关注，已经有QUTEST 听小编制与流行，打算弄个 nuwave 专听大编制与金属,1,2021-08-30 00:00:00.000000,音质
6823,6824,r6ToDL,44,音染也是HIFI的一种？,-1,2021-04-05 00:00:00.000000,音质
7465,7466,BczQLZ,30,感觉随身胆放挺危险，不知道电压多高？,0,2021-08-05 00:00:00.000000,配置
2014,2015,61Ebq1,36,固定钢圈还能调整？ ath不是一直不能调吗,0,2021-09-26 00:00:00.000000,其他


### 用飞浆添加情感分析结果

>**一、window10+anaconda3的安装命令：**
>
>conda install paddlepaddle==2.2.1 --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/Paddle/
>
>其他安装飞浆命令：[官网地址](https://www.paddlepaddle.org.cn/install/quick?docurl=/documentation/docs/zh/install/conda/windows-conda.html)
> 
>**二、安装预训练模型应用工具 PaddleHub**
>
>pip install paddlehub==2.0.0
>
>飞浆模型库地址：[官网地址](https://www.paddlepaddle.org.cn/hublist)
>
>飞浆情感分析模型介绍：[官网地址](https://www.paddlepaddle.org.cn/hubdetail?name=senta_bilstm&en_category=SentimentAnalysis)

In [50]:
import paddlehub as hub

# 加载模型
senta = hub.Module(name="senta_bilstm")

# 评论数据列表
test_text = df["content"].tolist()
# 模型返回的结果
results = senta.sentiment_classify(texts=test_text, use_gpu=False, batch_size=1)


[2022-01-12 17:49:37,431] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object
[2022-01-12 17:49:39,810] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


In [51]:
# 将返回的结果转为 dataframe 数据，并拼接到原始数据中
results_df = pd.DataFrame(results)

df2 = pd.concat([df,results_df],axis=1)

df2.sample(5)

### 数据清理

In [6]:
df2.subject.value_counts()

其他    7353
配置    1384
音质     597
价格     427
功能     102
外形      95
舒适      42
Name: subject, dtype: int64

In [40]:
# 去除情感状态未知的数据
df3 = df2[df2["sentiment_value"].isin([0,1])]

### 好评率

In [42]:
(df3.sentiment_value.value_counts(normalize=True)).map(lambda x:"{:.2%}".format(x))

0    73.83%
1    26.17%
Name: sentiment_value, dtype: object